In [1]:
import json, os
from copy import deepcopy
from pathlib import Path
import torch

results_output_path = "results"

In [2]:
call = "python main.py --model=hf-causal"


with open("run_arguments.json", "r") as f:
    args = json.load(f)


def fill_args(fillers: list, args: list):
    j = 0
    for i, val in enumerate(args):
        if not val:
            args[i] = fillers[j]
            j +=1
    return args

def mount_run_args(args: dict) -> str:
    out = []
    for key, val in args.items():
        if key == "--model_args='pretrained=":
             val = ','.join(val)
        if type(val) == list:
            val = ' '.join(val)
            args[key] = val
        out.append(key+val)
        
    return ' '.join(out)

def validate_output_path(model: str, task: str, base_path: str = "results") -> str:

    model = model.split('/')
    if len(model) > 1:
        model = model[-1]
    else:
        model = model[0]    
    
    base_path = base_path / Path(model)
    if not os.path.exists(base_path):
        os.makedirs(base_path)
    full_path = base_path / task
    with open(full_path, 'w') as f:
        pass
    return str(full_path)

def get_task_few_shot_num(args: dict, task: str) -> str:
    for t, val in args["few_shot"].items():
        found = task.find(t)
        if found != -1:
            return val

cheap_tasks = ["truthfulqa_mc"]

call_args = {"--machine=": "p3.2xlarge", 
             "--model=": "hf-causal",
             "--model_args=pretrained=": [None],
             "--tasks=" : [None],
              "--num_fewshot=" : [None],
              "--batch_size=" : ['2'],
              "--output_path=": [None]}

for model in args['models']:
    for task in args['cheap_tasks']:
        output_path = validate_output_path(model, task)
        new_args = deepcopy(call_args)
        new_args["--model_args=pretrained="] = fill_args([model], new_args["--model_args=pretrained="])
        new_args["--tasks="] = fill_args([task], new_args["--tasks="])
        num_fewshot = get_task_few_shot_num(args, task)
        new_args["--num_fewshot="] = fill_args([num_fewshot], new_args["--num_fewshot="])
        new_args["--output_path="] = fill_args([output_path], new_args["--output_path="])
        run_args = mount_run_args(new_args)
        print(run_args)
        !python main.py {run_args}
        torch.cuda.empty_cache()



--machine=p3.2xlarge --model=hf-causal --model_args=pretrained=meta-llama/Llama-2-7b-chat-hf --tasks=arc_challenge --num_fewshot=25 --batch_size=2 --output_path=results/Llama-2-7b-chat-hf/arc_challenge


Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /home/ubuntu/.cache/huggingface/token
Login successful
Selected Tasks: ['arc_challenge']
Using device 'cuda'
